### `Package using`

In [213]:
import cv2
import numpy as np
from itertools import groupby
import random as rd

### `Input`

In [214]:

def create_edges(positions):
    '''  Tạo cạnh từ danh sách đỉnh '''
    result = []
    for i in range(0, len(positions) - 1):
        item = positions[i]
        next_item = positions[i+1]
        if item[0] == next_item[0]:
            if item[1] < next_item[1]:
                # dọc thuận - done
                for i in range(item[1], next_item[1] + 1):
                    result.append([item[0], i])
            if item[1] > next_item[1]:
                # dọc ngược - done
                for i in range(next_item[1], item[1] + 1):
                    result.append([item[0], i])                
        
        elif item[1] == next_item[1]:    
            if item[0] < next_item[0]:
                # ngang thuan - done
                for i in range(item[0], next_item[0] + 1):
                    result.append([i, item[1]])         
            if item[0] > next_item[0]:
                # ngang nguoc - done
                for i in range(next_item[0], item[0] + 1):
                        result.append([i, item[1]])
    
    return list(set(tuple(x) for x in result))


# positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
#                       [430, 100], [520, 100], [520, 150],[600, 150], [600, 200], [520, 200]
#                       , [520, 440], [650, 440], [650, 630],
#                       [210, 630], [210, 530], [70, 530], [70, 420],
#                         [150, 420], [150, 140], [50, 140], [50, 40]]
# [ngang, dọc]
grid_size = [80, 80]
positions = [[50, 40], [580, 40], [580, 60], [430, 60], 
                      [430, 100], [520, 100], [520, 150],[600, 150], [600, 220], [620, 220],
                      [620, 180], [660, 180], [660, 280], 
                      [550,280], [550,220], [520, 220], 
                      [520, 440], [650, 440], [650, 630],
                      [210, 630], [210, 530], [70, 530], [70, 420], 
                      [150, 420], [150, 140], [50, 140], [50, 40]]
# positions = np.array([[60, 60], [270, 60], [270, 270], [60, 270], [60, 60]])
positions = [[50, 40], [150, 40], [150, 120], [350, 120], 
             [350, 60], [400, 60], [400, 200], [500, 200], 
             [500, 500], [50, 500], [50, 40]]



### `TẠO ĐƯỜNG BAO`

In [215]:
cell_edges = create_edges(positions)

## `TẠO ĐƯỜNG BAO CHO 4 PHẦN TÍNH TỪ ĐIỂM NẤT KỲ (VD: TRỌNG TÂM)`

In [216]:
trung_diem = [int(np.mean(np.array(cell_edges)[:,0])), int(np.mean(np.array(cell_edges)[:,1]))]
db1, db2, db3, db4 = [], [], [], []

for [x,y] in cell_edges:
        if x >= trung_diem[0] and y <= trung_diem[1]:
            db1.append([x, y]) 
        if x <= trung_diem[0] and y <= trung_diem[1]:
            db2.append([x, y])
        if x >= trung_diem[0] and y >= trung_diem[1]:
            db3.append([x, y])
        if x <= trung_diem[0] and y >= trung_diem[1]:
            db4.append([x, y])

mmx = [min(np.array(positions)[:,0]),max(np.array(positions)[:,0])]
mmy = [min(np.array(positions)[:,1]),max(np.array(positions)[:,1])]

for x in range(trung_diem[0], mmx[1]+1, 1):
    for y in range(trung_diem[1], mmy[0]-1, -1):
        if (x == trung_diem[0] or y == trung_diem[1]):  # là vị trí đường lưới
            arrX_item = [i for [i, j] in cell_edges if j == y]
            arrY_item = [j for [i, j] in cell_edges if i == x]
            if arrY_item and arrX_item:
                _x_ = [min(arrX_item), max(arrX_item)]
                _y_ = [min(arrY_item), max(arrY_item)]
                if ((x >= _x_[0]) and (x <= _x_[1])) and (y >= _y_[0] and y <= _y_[1]):  # bên trong
                    db1.append([x, y])

## `TẠO CẠNH CHO 4 PHÍA`

In [217]:
canh1 = []

for x in range(trung_diem[0], mmx[1]+1, 1):
    for y in range(trung_diem[1], mmy[0]-1, -1):
        if (x- trung_diem[0]) %  grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0:
            arrX_item = [i for [i, j] in db1 if y == j]
            arrY_item = [j for [i, j] in db1 if x == i]
            if arrY_item and arrX_item:
                mmx_ = [min(arrX_item), max(arrX_item)]
                mmy_ = [min(arrY_item), max(arrY_item)]
                if ((x >= mmx_[0]) and (x <= mmx_[1])) and (y >= mmy_[0] and y <= mmy_[1]):  # bên trong
                    canh1.append([x, y])

### `CHUẨN BỊ DỮ LIỆU CHO TẠO MA TRẬN`

## `TẠO MA TRẬN (1->Cạnh, 0->thịt, None->Vùng ngoài)`

In [218]:
cells1 = []
for x in range(trung_diem[0], mmx[1]+1, 1):
    arr = []
    for y in range(trung_diem[1], mmy[0]-1, -1):
        cell = [x,y,None, None]
        arrX_item = [i for [i, j] in db1 if y == j]
        arrY_item = [j for [i, j] in db1 if x == i]
        if arrY_item and arrX_item:
            mmx_ = [min(arrX_item), max(arrX_item)]
            mmy_ = [min(arrY_item), max(arrY_item)]
            if ((x >= mmx_[0]) and (x <= mmx_[1])) and (y >= mmy_[0] and y <= mmy_[1]):  # bên trong
                cell = [x, y, 0, None]
                if ((x- trung_diem[0]) %  grid_size[0] == 0 or (y - trung_diem[1]) % grid_size[1] == 0) or [x, y] in db1: 
                    cell = [x, y, 1, None]
        arr.append(cell)
    cells1.append(arr)

### `ĐÁNH DẤU GIÁ TRỊ "GROUP" CHO THỊT`

In [219]:
image = np.ones((700, 700, 3), np.uint8) * 255


result1 = cells1
# result2 = cells2
# result3 = cells3
# result4 = cells4
index = 1
cells_shape1 = np.array(result1).shape
# cells_shape2 = np.array(result2).shape
# cells_shape3 = np.array(result3).shape
# cells_shape4 = np.array(result4).shape

# 1
for i in range(0, cells_shape1[1]-1):
    for j in range(0, cells_shape1[0]-1):
        cell = result1[j][i]
        cell_trai = result1[j-1][i]
        cell_duoi = result1[j][i-1]
        cell_duoitrai = result1[j-1][i-1]
        if cell[2] == 0:
            if cell_duoitrai[2] == 1 and cell_duoi[2] == 1 and cell_trai[2] == 1:
                result1[j][i][3] = index
                index += 1
            if cell_trai[3] != None:
                result1[j][i][3] = cell_trai[3]
            if cell_duoi[3] != None:
                result1[j][i][3] = cell_duoi[3]

In [225]:
result1_copy = result1
shape_copy_1 = np.array(result1_copy).shape
res = []

for i in range(0, shape_copy_1[1]-1):
    for j in range(0, shape_copy_1[0]-1):
        cell = result1_copy[j][i]
        cell_trai = result1_copy[j-1][i]
        cell_phai = result1_copy[j+1][i]
        if (cell[3] != None and cell_phai[3] != None) and cell[3] - cell_phai[3] != 0:
                res.append([cell[3], cell_phai[3]])

# Sử dụng set() để loại bỏ các phần tử trùng lặp
lis = list(set(tuple(x) for x in res))

# In danh sách sau khi loại bỏ phần tử trùng lặp
print("Danh sách sau khi loại bỏ phần tử trùng lặp:", lis)


Danh sách sau khi loại bỏ phần tử trùng lặp: []


### `Hợp nhất các nhóm khác nhau trong 1 mảnh`

In [221]:
for (grx, gry) in lis:
    for i in range(0, cells_shape1[1]-1):
        for j in range(0, cells_shape1[0]-1):
            cell = result1[j][i]
            if cell[3] == gry:
                result1[j][i][3] = grx

### `TẠO DANH SÁCH GOM NHÓM CÁC ĐỈNH CÓ PHẦN THỊT CÙNG 1 NHÓM`

In [222]:
image = np.ones((700, 700, 3), np.uint8) * 255

# gom nhóm các đỉnh của mảnh nhỏ
result_11 = []
result_22 = []
result_33 = []
result_44 = []

cells_shape1 = np.array(result1).shape

### 1
for i in range(0, cells_shape1[0]):
    for j in range(0, cells_shape1[1]):
        cell = result1[i][j]
        
        if cell[2] == 0:
            cell_trai = result1[i-1][j]
            cell_phai = result1[i+1][j]
            cell_duoi = result1[i][j-1]
            cell_tren = result1[i][j+1]
            cell_duoiTrai = result1[i-1][j-1]
            cell_duoiPhai = result1[i+1][j-1]
            cell_trenTrai = result1[i-1][j+1]
            cell_trenPhai = result1[i+1][j+1]
            if (i > 2 and j > 2) and (i < cells_shape1[0]-2 and j < cells_shape1[1] - 2):
                if (result1[i+1][j+2] != None and 
                    result1[i+2][j+1] != None and
                    result1[i-1][j+2] != None and 
                    result1[i-2][j+1] != None and
                    result1[i-1][j-2] != None and 
                    result1[i-2][j-1] != None and
                    result1[i+1][j-2] != None and 
                    result1[i+2][j-1] != None):
                    
                    #           |
                    # tren phai x__
                    cell_tren_cua_trenPhai = result1[i+1][j+2]
                    cell_phai_cua_trenPhai = result1[i+2][j+1]
                    #   |
                    # __x tren trai
                    cell_tren_cua_trenTrai = result1[i-1][j+2]
                    cell_trai_cua_trenTrai = result1[i-2][j+1]
                    # __x duoi trai
                    #   |
                    cell_duoi_cua_duoiTrai = result1[i-1][j-2]
                    cell_trai_cua_duoiTrai = result1[i-2][j-1]
                    # duoi phai x__
                    #           |
                    cell_duoi_cua_duoiPhai = result1[i+1][j-2]
                    cell_phai_cua_duoiPhai = result1[i+2][j-1]
                    
                    if cell_duoiTrai[2] == 1 and cell_duoi[2] == 1 and cell_trai[2] == 1: # 1
                        result_11.append([cell_duoiTrai[0], cell_duoiTrai[1], cell_duoiTrai[2] , cell[3]])
                        x,y,val,gr = cell_duoiTrai
                        cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                    elif cell_duoiPhai[2] == 1 and cell_duoi[2] == 1 and cell_phai[2] == 1: # 2
                        result_11.append([cell_duoiPhai[0], cell_duoiPhai[1], cell_duoiPhai[2] , cell[3]])
                        x,y,val,gr = cell_duoiPhai
                        cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                    elif cell_trenTrai[2] == 1 and cell_tren[2] == 1 and cell_trai[2] == 1: # 3
                        result_11.append([cell_trenTrai[0], cell_trenTrai[1], cell_trenTrai[2] , cell[3]])
                        x,y,val,gr = cell_trenTrai
                        cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                    elif cell_trenPhai[2] == 1 and cell_tren[2] == 1 and cell_phai[2] == 1: # 4
                        result_11.append([cell_trenPhai[0], cell_trenPhai[1], cell_trenPhai[2] , cell[3]])
                        x,y,val,gr = cell_trenPhai
                        cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                    
                    elif cell_trenPhai[2] == 1 and cell_tren_cua_trenPhai[2] == 1 and cell_phai_cua_trenPhai[2] == 1: # 5
                        result_11.append([cell_trenPhai[0], cell_trenPhai[1], cell_trenPhai[2] , cell[3]])
                        x,y,val,gr = cell_trenPhai
                        cv2.line(image, (x, y), (x, y), (255,0,0), 3)
                    elif cell_trenTrai[2] == 1 and cell_tren_cua_trenTrai[2] == 1 and cell_trai_cua_trenTrai[2] == 1: # 6
                        result_11.append([cell_trenTrai[0], cell_trenTrai[1], cell_trenTrai[2] , cell[3]])
                        x,y,val,gr = cell_trenTrai
                        cv2.line(image, (x, y), (x, y), (255,0,0), 3)
                    elif cell_duoiPhai[2] == 1 and cell_duoi_cua_duoiPhai[2] == 1 and cell_phai_cua_duoiPhai[2] == 1: # 7
                        result_11.append([cell_duoiPhai[0], cell_duoiPhai[1], cell_duoiPhai[2] , cell[3]])
                        x,y,val,gr = cell_duoiPhai
                        cv2.line(image, (x, y), (x, y), (255,0,0), 3)
                    elif cell_duoiTrai[2] == 1 and cell_duoi_cua_duoiTrai[2] == 1 and cell_trai_cua_duoiTrai[2] == 1: # 8
                        result_11.append([cell_duoiTrai[0], cell_duoiTrai[1], cell_duoiTrai[2] , cell[3]])
                        x,y,val,gr = cell_duoiTrai
                        cv2.line(image, (x, y), (x, y), (255,0,0), 3)
            else:
                if cell_duoiTrai[2] == 1 and cell_duoi[2] == 1 and cell_trai[2] == 1: # 1
                    result_11.append([cell_duoiTrai[0], cell_duoiTrai[1], cell_duoiTrai[2] , cell[3]])
                    x,y,val,gr = cell_duoiTrai
                    cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                elif cell_duoiPhai[2] == 1 and cell_duoi[2] == 1 and cell_phai[2] == 1: # 2
                    result_11.append([cell_duoiPhai[0], cell_duoiPhai[1], cell_duoiPhai[2] , cell[3]])
                    x,y,val,gr = cell_duoiPhai
                    cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                elif cell_trenTrai[2] == 1 and cell_tren[2] == 1 and cell_trai[2] == 1: # 3
                    result_11.append([cell_trenTrai[0], cell_trenTrai[1], cell_trenTrai[2] , cell[3]])
                    x,y,val,gr = cell_trenTrai
                    cv2.line(image, (x, y), (x, y), (255,0,255), 3)
                elif cell_trenPhai[2] == 1 and cell_tren[2] == 1 and cell_phai[2] == 1: # 4
                    result_11.append([cell_trenPhai[0], cell_trenPhai[1], cell_trenPhai[2] , cell[3]])
                    x,y,val,gr = cell_trenPhai
                    cv2.line(image, (x, y), (x, y), (255,0,255), 3)
        elif cell[2] == 1:
            x,y,val,gr = cell
            cv2.line(image, (x, y), (x, y), (0,0,0), 1)

In [226]:
result_all = result_11
result_all = sorted(result_all, key=lambda x: x[3])
result = [list(group) for key, group in groupby(result_all, key= lambda x: x[3])]
print('danh sách tọa độ các mảnh\nTổng số mảnh:', len(result))
for item in result:
    print(len(item),item)

danh sách tọa độ các mảnh
Tổng số mảnh: 6
4 [[262, 275, 1, 1], [262, 195, 1, 1], [342, 275, 1, 1], [342, 195, 1, 1]]
6 [[342, 275, 1, 2], [342, 195, 1, 2], [400, 200, 1, 2], [400, 195, 1, 2], [422, 275, 1, 2], [422, 200, 1, 2]]
4 [[422, 275, 1, 3], [422, 200, 1, 3], [500, 275, 1, 3], [500, 200, 1, 3]]
4 [[262, 195, 1, 4], [262, 120, 1, 4], [342, 195, 1, 4], [342, 120, 1, 4]]
6 [[342, 195, 1, 5], [342, 120, 1, 5], [350, 120, 1, 5], [350, 115, 1, 5], [400, 195, 1, 5], [400, 115, 1, 5]]
4 [[350, 115, 1, 6], [350, 60, 1, 6], [400, 115, 1, 6], [400, 60, 1, 6]]


### `TRỰC QUAN HÓA KẾT QUẢ TỔNG KHUNG HÌNH`

In [227]:

image = np.ones((700, 700, 3), np.uint8) * 255

# Lọc lây toàn bộ thịt
array_thit = [item for item in np.array(result1).reshape(-1,4).tolist() if item[3] != None]
# Sắp xếp theo group
r_1 = sorted(array_thit, key= lambda x: x[3])
# gom thịt có group giống nhau thành 1 danh sách con
groups_thit = [list(group) for key, group in groupby(r_1, key=lambda x: x[3])]
# Sắp xếp mảng thịt có số lượng từ lớn đến bé
groups_thit = sorted(groups_thit, key=len, reverse=True)

check = len(groups_thit[0])
color = (255,0,0)
for arr in groups_thit:
    if check - len(arr) != 0:
        color = (rd.randint(0,250), rd.randint(0,250), rd.randint(200,255))
        check = len(arr)
        
    for [x,y,val,gr] in arr:
        if val == 0:
            cv2.line(image, (x,y), (x,y), color,1)

for i in range(0, cells_shape1[0]):
    for j in range(0, cells_shape1[1]):
        x,y,val,gr = result1[i][j]
        if val == 1:
            cv2.line(image, (x,y), (x,y), (0,0,0),1)
        elif val == None:
            cv2.line(image, (x,y), (x,y), (200,200,200),1)

[cv2.line(image, (x,y), (x,y), (0,0,0),1) for [x,y] in cell_edges]

while True:
    cv2.imshow('DatGach', image)
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()

# PHÂN LOẠI VÀ ĐẾM SỐ CÁC HÌNH GIỐNG NHAU